# 3 API 參數解析與錯誤處理

## 3-1 事前準備

請依照第 2 章說明上傳 .env 檔

In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

In [ ]:
!pip install openai
import openai

## 3-2 控制生成訊息與 token 數量

### 指定生成的訊息數量 - n

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "你好"}],
  n=2
)

print(reply)

for choice in reply["choices"]:
    print(choice['index'], choice["message"]["content"])

### 設定詞彙黑名單 - stop

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "你好"}],
  stop=['好']
)

print(reply["choices"][0]["message"]["content"])
print(reply["choices"][0]["finish_reason"])

### 設定回覆語句的 tokens 數量上限 - max_tokens

In [ ]:
reply = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"user", "content": "您好!"}
    ],
    max_tokens = 5
)

print(reply["choices"][0]["message"]["content"])
print(reply['choices'][0]["finish_reason"])
print(reply['usage']['completion_tokens'])

In [ ]:
!pip install tiktoken
!pip install ansicolors
!git clone https://github.com/FlagTech/flagtokenizer.git
from flagtokenizer import tokenizer

In [ ]:
tokenizer("您好！有什")

超過模型限制的 tokens 數

In [ ]:
reply = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"user", "content": "你好"}
    ],
    max_tokens = 4090
)

In [ ]:
reply = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [{
        "role":"user",
            "content": "請加上編號重複以下這句話 1000 次："
            "我喜歡 AI 人工智慧"
    }],
)

In [ ]:
print(reply['choices'][0]['message']['content'][:100])
print('...')
print(reply['choices'][0]['message']['content'][-100:])

In [ ]:
print(len(tokenizer(
    "請加上編號重複以下這句話 1000 次："
    "我喜歡 AI 人工智慧",
    show=False
)))
print(reply['usage']['prompt_tokens'])
print(reply['usage']['completion_tokens'])
print(reply['choices'][0]['finish_reason'])

## 3-3 控制回覆內容的變化性

### 讓回覆更具彈性 - temperature

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "嗨！"}],
  temperature=0,
  n=2
)

for choice in reply["choices"]:
    print(choice['index'], choice["message"]["content"])

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "嗨！"}],
  temperature=2,
  n=2
)

for choice in reply["choices"]:
    print(choice['index'], choice["message"]["content"])

### 控制詞彙的豐富度 - top_p

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "嗨！"}],
  top_p=0,
  n=2
)

for choice in reply["choices"]:
    print(choice['index'], choice["message"]["content"])

### 控制詞彙的重複性 - presence_penalty 與 frequency_penalty

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{
    "role": "user",
    "content": "台北是什麼樣的城市？"
  }],
  temperature=1,
  presence_penalty=-2,
)

print(reply['choices'][0]["message"]["content"])

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{
      "role": "user",
      "content": "台北是什麼樣的城市？"}],
  temperature=1,
  frequency_penalty=-2,
)

print(reply['choices'][0]["message"]["content"])

### 調整特定 token 的分數 - logi-bias


In [ ]:
tokenizer('你好', show=False)

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "你好"}],
  temperature=1,
  logit_bias={
      53901: -100,
      57668: -100
  },
)

print(reply['choices'][0]["message"]["content"])

In [ ]:
tokenizer('哈', show=False)

In [ ]:
reply = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": "你好"}],
  temperature=1,
  logit_bias={
      99771: 100,
  },
)

print(reply['choices'][0]["message"]["content"])

## 3-4 串流輸出

### 可循序傳回結果的生成器 (generator) - stream


In [ ]:
replies = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{
    "role": "user",
    "content": "你好"
  }],
  stream=True,
)

for reply in replies:
    print(reply)

In [ ]:
replies = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{
    "role": "user",
    "content": "台北是什麼樣的城市？"
  }],
  stream=True
)

for reply in replies:
    if 'content' in reply['choices'][0]['delta']:
        print(reply['choices'][0]['delta']['content'], end='')

### 串流多個語句

In [ ]:
replies = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{
    "role": "user",
    "content": "你好"
  }],
  stream=True,
  n=2
)

for reply in replies:
    if 'content' in reply['choices'][0]['delta']:
        print(reply['choices'][0]['delta']['content'], end='')

## 3-5 錯誤處理與使用限制

### 使用例外機制處理錯誤

In [ ]:
try:
    reply = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role":"user", "content": "你住的地方很亮嗎？"}
        ],
        max_tokens = 4096
    )

    print(reply["choices"][0]["message"]["content"])
except openai.OpenAIError as err:
    print("錯誤種類", err.error.type)
    print("錯誤訊息", err.error.message)